In [420]:
import numpy as np
import pandas as pd
import requests, os
import time
import random
import seaborn as sns
# import csv
import datetime as dt
import regex as re
# import json
# import os
# import statistics
pd.options.display.max_columns = 30
pd.options.display.max_rows = 200

In [434]:
"".split(', ')

['']

In [422]:
store_df = pd.read_csv('data/raw/store_data.csv',index_col=0)
english_fixes_df = pd.read_csv('data/raw/store_data_english_fixes.csv',index_col=0)
store_df.update(english_fixes_df)

store_df = store_df[~store_df['name'].isnull()]
store_df = store_df[store_df['type']=='game']
store_df = store_df.drop(columns=['type'])
store_df.dropna(subset=['supported_languages'], inplace=True)
store_df['required_age'] = store_df['required_age'].astype(str)
store_df['required_age'] = store_df['required_age'].str.replace('\.0','').str.replace('\+','').str.replace('1818','18').astype(int)
store_df = store_df[~(store_df['supported_languages'].str.contains('(\\r\\n|\#)',regex=True))]
store_df = store_df.drop(index=[601,12472])



store_df['supported_languages'] = (store_df['supported_languages']
                                   .str.replace(',',', ')
                                   .str.replace(',  ',', ')
                                   .str.replace(' \(full audio\)','*')
                                   .str.replace(' \(text only\)','')
                                   .str.replace('\;','')
                                   .str.replace('  ','')
                                   .str.replace('\<br\>\<strong\>\*\<\/strong\>languages with full audio support','')
                                   .str.replace('\<strong\>\*\<\/strong\>','*')
                                   .str.replace(',,',',')
                                  )



store_df['supported_languages_audio'] = store_df['supported_languages'].str.split(', ')
store_df['supported_languages'] = store_df['supported_languages'].str.replace('*','')
store_df['release_date'] = pd.to_datetime(store_df['release_date'],errors='coerce')
store_df.dropna(subset=['release_date'], inplace=True)
store_df['age_days'] = (dt.datetime.now() - store_df['release_date']).dt.days
store_df['publishers'] = (store_df['publishers']
                          .str.replace(', *\.*(i\.*n\.*c|l\.*t\.*d|l\.*l\.*c|s\.r\.o|s\.c|l\.*d\.*a|co\.)\.*', '', flags=re.IGNORECASE, regex=True)
                          .str.replace(', *\.*(ACE|SL|a\.s)\.*', '', regex=True)
                          .str.replace(',[^,]*\((mac|linux)( *,*/* *(mac|linux))*\)','', flags=re.IGNORECASE, regex=True)
                          .str.replace(',,',',', flags=re.IGNORECASE, regex=True)
                          .str.replace(', *$', '', flags=re.IGNORECASE, regex=True)
                         )
store_df['developers'] = (store_df['developers']
                          .str.replace(', *\.*(i\.*n\.*c|l\.*t\.*d|l\.*l\.*c|s\.r\.o|s\.c|l\.*d\.*a|co\.)\.*', '', flags=re.IGNORECASE, regex=True)
                          .str.replace(', *\.*(ACE|SL|a\.s)\.*', '', regex=True)
                          .str.replace(',[^,]*\((mac|linux)( *,*/* *(mac|linux))*\)','', flags=re.IGNORECASE, regex=True)
                          .str.replace(',,',',', flags=re.IGNORECASE, regex=True)
                          .str.replace(', *$', '', flags=re.IGNORECASE, regex=True)
                         )

store_df = store_df.dropna(subset=['developers','categories','genres'])

def strong_only(x):
    return [y[:-1] for y in x if y[-1]=='*']

store_df['supported_languages_audio'] = store_df['supported_languages_audio'].apply(strong_only)
store_df['supported_languages_audio'] = store_df['supported_languages_audio'].apply(lambda x: ', '.join(x))


store_df['controller_support'] = store_df['controller_support'].apply(lambda x: 1 if x=='full' else 0).astype(int)
store_df['is_free'] = store_df['is_free'].apply(lambda x: 1 if x==True else 0).astype(int)
print(len(store_df))

# len(
# store_df['developers'][store_df['developers'].str.contains('$', flags=re.IGNORECASE, regex=True)].head(100)


store_df[(store_df['supported_languages'].str.contains('DutchEnglish',regex=True))]

# )

/Users/matteofortier/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/matteofortier/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


43531


,name,steam_appid,required_age,is_free,controller_support,supported_languages,developers,publishers,platforms,metacritic,categories,genres,recommendations,release_date,supported_languages_audio,age_days


In [423]:
spy_df = pd.read_csv('data/raw/app_data.csv', index_col=0)
spy_df['owners'] = spy_df['owners'].str.replace('\ ','').str.replace('\,','').str.split('\.\.')
spy_df['owners'] = spy_df['owners'].apply(lambda x: x[1])
spy_df['owners'] = spy_df['owners'].astype(int)
spy_df

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,ccu,languages,genre,tags,tag_values
0,570,Dota 2,Valve,Valve,NaN,1290431,244850,0,200000000,34777,1456,947,678,0.0,0.0,0.0,608470,"English, Bulgarian, Czech, Danish, Dutch, Finn...","Action, Free to Play, Strategy","Free to Play, MOBA, Multiplayer, Strategy, e-s...","57948, 19227, 14461, 13507, 10938, 10351, 7711..."
1,730,Counter-Strike: Global Offensive,"Valve, Hidden Path Entertainment",Valve,NaN,5059612,684936,0,100000000,28812,697,6898,294,0.0,0.0,0.0,696817,"English, Czech, Danish, Dutch, Finnish, French...","Action, Free to Play","FPS, Shooter, Multiplayer, Competitive, Action...","86666, 62164, 59376, 50866, 45505, 44432, 4126..."
2,578080,PUBG: BATTLEGROUNDS,"KRAFTON, Inc.","KRAFTON, Inc.",NaN,963190,805610,0,100000000,25452,656,11829,178,2999.0,2999.0,0.0,341397,"English, Korean, Simplified Chinese, French, G...","Action, Adventure, Massively Multiplayer","Survival, Shooter, Multiplayer, Battle Royale,...","13360, 11152, 9664, 9235, 7325, 7003, 6484, 54..."
3,440,Team Fortress 2,Valve,Valve,NaN,747552,49308,0,100000000,7929,1569,341,263,0.0,0.0,0.0,87479,"English, Danish, Dutch, Finnish, French, Germa...","Action, Free to Play","Free to Play, Hero Shooter, Multiplayer, FPS, ...","61038, 60376, 21537, 14400, 11738, 11281, 8849..."
4,304930,Unturned,Smartly Dressed Games,Smartly Dressed Games,NaN,404354,38376,0,50000000,3999,1984,310,957,0.0,0.0,0.0,36802,English,"Action, Adventure, Casual, Free to Play, Indie","Free to Play, Survival, Zombies, Open World Su...","36020, 15727, 14720, 14425, 13861, 12372, 9336..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44849,858730,Yoke Light,Kotoshiro,Kotoshiro,NaN,0,0,0,20000,0,0,0,0,99.0,99.0,0.0,0,English,"Action, Casual, Indie",NaN,NaN
44850,721460,Xeno Time Inception,Xeno Gaming LLC,Xeno Gaming LLC,NaN,0,0,0,20000,0,0,0,0,0.0,0.0,0.0,0,English,"Action, Adventure, Indie, RPG, Early Access",NaN,NaN
44851,433700,Carpe Lucem - Seize The Light VR,Hammer Labs,Application Systems Heidelberg,NaN,41,5,0,20000,0,0,0,0,899.0,1499.0,40.0,0,English,"Casual, Indie, Strategy","Casual, VR, Indie, Strategy, Puzzle, Great Sou...","43, 43, 42, 32, 23, 23, 22, 13, 13, 12, 12, 12..."
44852,650980,Naklua VR,Fly Dream Dev,Fly Dream Dev,NaN,1,3,0,20000,0,0,0,0,99.0,199.0,50.0,0,English,"Adventure, Casual, Indie, Sports, Early Access","Indie, Early Access, Casual, Adventure, Sports...","21, 21, 21, 20, 20, 5"


In [424]:
combined = pd.merge(spy_df.drop(columns=['name','genre','developer','publisher','score_rank','languages']),store_df,right_on='steam_appid',left_on='appid').set_index('appid')

In [425]:
combined = combined[['name', 'developers', 'publishers', 'platforms', 'categories', 'genres', 'tags', 'tag_values',  
 'positive', 'negative', 'userscore', 'owners',
       'average_forever', 'average_2weeks', 'median_forever', 'median_2weeks', 'ccu', 
       'is_free', 'price', 'initialprice', 'discount',
       'required_age', 'controller_support', 'supported_languages', 'supported_languages_audio',
       'metacritic', 'recommendations',
       'release_date','age_days']]

In [426]:
combined.to_csv('data/combined.csv')